In [6]:
import pandas as pd
from ipywidgets import widgets
from IPython.display import display
import datetime as dt
pd.options.display.max_colwidth = 10000
pd.options.display.max_rows = 10000

def append_row(accounts_df, date_str, col_name, value, description):
    row_idx = len(accounts_df)
    date_obj = dt.datetime.strptime(date_str,"%Y-%m-%d 00:00:00")
    accounts_joined.loc[row_idx, "Date"] = date_obj
    accounts_joined.loc[row_idx, "Description"] = description
    accounts_joined.loc[row_idx, col_name] = value

all_payments = pd.read_csv("/Users/johnclark/Documents/Finances/All_Payment_Methods020122.csv")
end_of_year = pd.read_csv("/Users/johnclark/Documents/Finances/Year End All Payment Methods020122.csv",parse_dates=[2], header=0)
end_of_year["Amount_value"] = end_of_year["Amount"].apply(lambda x: float(x[1:].replace(",","")) if x.startswith("$") else -float(x[2:].replace(",","")) if x.startswith("-$") else x)

no_credit_card = end_of_year[end_of_year["Payee"] != "WELLS FARGO CREDIT CARD"]
no_wells_credit_rec_trans = end_of_year[(end_of_year["Payee"] != "WELLS FARGO CREDIT CARD") & (~end_of_year["Description"].str.startswith("RECURRING TRANSFER")) \
                                        & (~end_of_year["Description"].str.startswith("ONLINE TRANSFER TO CLARK J SAVINGS XXXXXX5751")) \
                                        & (~end_of_year["Description"].str.startswith("ONLINE TRANSFER TO CLARK J CHECKING XXXXXX9873"))]

years = [2022] #[2020, 2021, 2022]
accounts_joined = pd.DataFrame([], columns=["Date","Amount","Unk","CheckNum","Description"])
for year in years:
    utility_check = pd.read_csv("/Users/johnclark/Documents/Finances/Utility{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
    utility_check["account_name"] = "utility"
    umbrella_check = pd.read_csv("/Users/johnclark/Documents/Finances/Umbrella{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
    umbrella_check["account_name"] = "umbrella"
    savings_check = pd.read_csv("/Users/johnclark/Documents/Finances/Savings{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
    savings_check["account_name"] = "savings"
    primary_check = pd.read_csv("/Users/johnclark/Documents/Finances/Primary{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
    primary_check["account_name"] = "primary"
    platinum_card = pd.read_csv("/Users/johnclark/Documents/Finances/PlatinumCard{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
    platinum_card["account_name"] = "platinum"
    cashwise_card = pd.read_csv("/Users/johnclark/Documents/Finances/CashWiseCard{}.csv".format(year), header=None, names=["Date","Amount","Unk","CheckNum","Description"], parse_dates=[0])
    cashwise_card["account_name"] = "cashwise"

    accounts_joined = accounts_joined.append(utility_check)
    accounts_joined = accounts_joined.append(umbrella_check)
    accounts_joined = accounts_joined.append(savings_check)
    accounts_joined = accounts_joined.append(primary_check)
    # accounts_joined = accounts_joined.append(platinum_card)
    # accounts_joined = accounts_joined.append(cashwise_card)

accounts_joined.reset_index(inplace=True, drop=True)

#assert len(accounts_joined) == len(utility_check) + len(umbrella_check) + len(savings_check) + len(primary_check)# + len(platinum_card) + len(cashwise_card)

accounts_joined["spending"] = accounts_joined["Amount"].apply(lambda x: x if x<0.0 else 0.0)
accounts_joined["income"] = accounts_joined["Amount"].apply(lambda x: x if x>0.0 else 0.0)

exclude_desc = {"FID BKG SVC LLC MONEYLINE 210402 X49800767 O0TLA JOHN P CLARK","RECURRING TRANSFER TO CLARK",
                "ONLINE TRANSFER TO CLARK J CHECKING XXXXXX7135", 
                "ONLINE TRANSFER TO CLARK J CHECKING XXXXXX9865", 
                "ONLINE TRANSFER TO CLARK J CHECKING XXXXXX9873", 
                "ONLINE TRANSFER TO CLARK J SAVINGS XXXXXX5751", 
                "ONLINE TRANSFER FROM CLARK J CHECKING XXXXXX7135",
                "ONLINE TRANSFER FROM CLARK J CHECKING XXXXXX9865",
                "ONLINE TRANSFER FROM CLARK J CHECKING XXXXXX9873",
                "ONLINE TRANSFER FROM CLARK J SAVINGS XXXXXX5751",
                "ONLINE TRANSFER FROM CLARK J",
                "ONLINE TRANSFER TO CLARK J"}
tithe_desc_starts = {"RURAL INDIA MINI", "BILL PAY Rural India Ministries ", "BILL PAY Church of the Open Door",
                     "CHECK # 136","BUSINESS TO BUSINESS ACH www.gracepres.co", 
                     "CHECK # 198",
                     "BILL PAY Church of the Resurrection",
                     "Flagstaff Christ", "FLAGSTAFF CHRIST",
                     "Mount Calvary"}
tax_desc_starts = {"CHECK # 132",
                   "CHECK # 131",
                   "FRANCHISE TAX BD CASTTAXRFD",
                   "IRS USATAXPYMT"}
shoffice_starts = {"CHECK # 129","DEPOSITED OR CASHED CHECK # 128","CASHED CHECK # 127"}
dirtbike_starts = {"CHECK # 195","CHECK # 196"}

accounts_joined["chase"] = accounts_joined.apply(lambda r: r["Amount"] if r["Description"].startswith("CHASE") else 0.0, raw=False, axis=1)
accounts_joined["tithe"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in tithe_desc_starts if r["Description"].startswith(td)]) > 0  else 0.0, raw=False, axis=1)
accounts_joined["taxes"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in tax_desc_starts if r["Description"].startswith(td)]) > 0  else 0.0, raw=False, axis=1)
accounts_joined["taxes_spend"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in tax_desc_starts if r["Description"].startswith(td)]) > 0 and r["Amount"] < 0.0  else 0.0, raw=False, axis=1)
accounts_joined["shoffice"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in shoffice_starts if r["Description"].startswith(td)]) > 0 and r["Amount"] < 0.0  else 0.0, raw=False, axis=1)
accounts_joined["dirtbikes"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in dirtbike_starts if r["Description"].startswith(td)]) > 0 and r["Amount"] < 0.0  else 0.0, raw=False, axis=1)
accounts_joined["apple"] = accounts_joined.apply(lambda r: r["Amount"] if r["Description"].startswith("APPLE INC. PAYROLL") else 0.0, raw=False, axis=1)

accounts_joined["base_spend_adjustment"] = 0.0
append_row(accounts_joined, "2022-05-09 00:00:00", "base_spend_adjustment", -692.20, description="Sig 365X") #
append_row(accounts_joined, "2022-05-09 00:00:00", "base_spend_adjustment", -2838.70, description="Stacato P") #

append_row(accounts_joined, "2022-02-27 00:00:00", "base_spend_adjustment", -2573.72, description="Avis mistake charge") #

append_row(accounts_joined, "2022-02-15 00:00:00", "base_spend_adjustment", -749.50, description="Alaska air") #
append_row(accounts_joined, "2022-02-15 00:00:00", "base_spend_adjustment", -749.50, description="Alaska air") #
append_row(accounts_joined, "2022-02-15 00:00:00", "base_spend_adjustment", -749.50, description="Alaska air") #
append_row(accounts_joined, "2022-02-15 00:00:00", "base_spend_adjustment", -749.50, description="Alaska air") #
append_row(accounts_joined, "2022-02-15 00:00:00", "base_spend_adjustment", -749.50, description="Alaska air") #
append_row(accounts_joined, "2022-02-15 00:00:00", "base_spend_adjustment", -749.50, description="Alaska air") #

append_row(accounts_joined, "2022-02-10 00:00:00", "base_spend_adjustment", -3458.58, description="Alaska turo car") # 
append_row(accounts_joined, "2022-02-05 00:00:00", "base_spend_adjustment", -5560.00, description="Alaska royal star train") #

append_row(accounts_joined, "2022-02-04 00:00:00", "base_spend_adjustment", -477.18, description="Alaska Grand Denali Lodge") #
append_row(accounts_joined, "2022-02-04 00:00:00", "base_spend_adjustment", -452.93, description="Alaska Grand Denali Lodge") #
append_row(accounts_joined, "2022-02-04 00:00:00", "base_spend_adjustment", -460.93, description="Alaska Grand Denali Lodge") #

append_row(accounts_joined, "2022-01-08 00:00:00", "base_spend_adjustment", -364.62, description="Sportsmans pelican case") #
append_row(accounts_joined, "2021-12-20 00:00:00", "base_spend_adjustment", -309.96, description="Southwest for Tara") #

append_row(accounts_joined, "2021-12-17 00:00:00", "base_spend_adjustment", -869.16, description="Suppressor") #
append_row(accounts_joined, "2021-12-17 00:00:00", "base_spend_adjustment", -2117.98, description="Herman miller chair") #

append_row(accounts_joined, "2021-11-26 00:00:00", "base_spend_adjustment", -375.47, description="Black Barts dinner") #
append_row(accounts_joined, "2021-11-08 00:00:00", "base_spend_adjustment", -107.99, description="Push to talk") #
append_row(accounts_joined, "2021-10-10 00:00:00", "base_spend_adjustment", -687.54, description="Anniversary El Quoquistador") #
append_row(accounts_joined, "2021-09-29 00:00:00", "base_spend_adjustment", -1916.00, description="10 yr Anniversary earings") #

append_row(accounts_joined, "2021-08-25 00:00:00", "base_spend_adjustment", -403.17, description="Moms Iphone") #
append_row(accounts_joined, "2021-08-25 00:00:00", "base_spend_adjustment", -2240.27, description="new Carpet final payment") #

append_row(accounts_joined, "2021-08-06 00:00:00", "base_spend_adjustment", -500.38, description="Watermans dinner for Tara and Mikes anniversary") #
append_row(accounts_joined, "2021-07-31 00:00:00", "base_spend_adjustment", -653.99, description="Papas ipad") #
append_row(accounts_joined, "2021-07-30 00:00:00", "base_spend_adjustment", -2619.25, description="Hannahs new laptop") #
append_row(accounts_joined, "2021-07-13 00:00:00", "base_spend_adjustment", -580.04, description="Sportsmans Hannahs scope") #
append_row(accounts_joined, "2021-07-11 00:00:00", "base_spend_adjustment", -952.03, description="Sportsmans Hannahs gun") #
append_row(accounts_joined, "2021-07-08 00:00:00", "base_spend_adjustment", -2072.00, description="New carpet") #
append_row(accounts_joined, "2021-06-24 00:00:00", "base_spend_adjustment", -378.59, description="El conquist") #
append_row(accounts_joined, "2021-06-20 00:00:00", "base_spend_adjustment", -250.00, description="Princess cruise") #

append_row(accounts_joined, "2021-06-18 00:00:00", "base_spend_adjustment", -375.00, description="Princess cruise") #
append_row(accounts_joined, "2021-06-18 00:00:00", "base_spend_adjustment", -500.00, description="Princess cruise") #
append_row(accounts_joined, "2021-06-18 00:00:00", "base_spend_adjustment", -250.00, description="Princess cruise") #

append_row(accounts_joined, "2021-03-29 00:00:00", "base_spend_adjustment", -880.00, description="hunting rifle") #

append_row(accounts_joined, "2021-02-23 00:00:00", "base_spend_adjustment", -1148.49, description="Dirt bike repair") #
append_row(accounts_joined, "2021-02-23 00:00:00", "base_spend_adjustment", -985.94, description="Dirt bike repair") #

append_row(accounts_joined, "2021-02-05 00:00:00", "base_spend_adjustment", -1182.10, description="Shoffice charge on cashwise") #
append_row(accounts_joined, "2021-02-05 00:00:00", "base_spend_adjustment", -46.25, description="Shoffice charge on cashwise") #
append_row(accounts_joined, "2021-02-05 00:00:00", "base_spend_adjustment", -231.14, description="Shoffice charge on cashwise") #
append_row(accounts_joined, "2021-02-05 00:00:00", "base_spend_adjustment", -61.71, description="Shoffice charge on cashwise") #
append_row(accounts_joined, "2021-02-05 00:00:00", "base_spend_adjustment", -60.97, description="Shoffice charge on cashwise") #
na_maps = {col_name:0.0 for col_name in accounts_joined.columns}
na_maps["Description"] = ""
accounts_joined.fillna(na_maps, inplace=True)


accounts_joined["filtered_amount"] = accounts_joined.apply(lambda r: r["Amount"] if len([1 for td in exclude_desc if r["Description"].startswith(td)]) == 0 else 0.0, raw=False, axis=1)
accounts_joined["filtered_income"] = accounts_joined.apply(lambda r: r["Amount"] if r["Amount"]>0.0 and len([1 for td in exclude_desc if r["Description"].startswith(td)]) == 0 else 0.0, raw=False, axis=1)

accounts_joined["full_spending"] = accounts_joined.apply(lambda r: r["Amount"] if r["Amount"]<0.0 and len([1 for td in exclude_desc if r["Description"].startswith(td)]) == 0 else 0.0, raw=False, axis=1)
accounts_joined["full_spending"] = accounts_joined["full_spending"] - accounts_joined["taxes_spend"]

accounts_joined["base_spending"] = accounts_joined.apply(lambda r: r["Amount"] if r["Amount"]<0.0 and len([1 for td in exclude_desc if r["Description"].startswith(td)]) == 0 else 0.0, raw=False, axis=1)
accounts_joined["base_spending"] = accounts_joined["base_spending"] - accounts_joined["taxes_spend"] - accounts_joined["shoffice"] - accounts_joined["dirtbikes"] - accounts_joined["base_spend_adjustment"]

accounts_joined["comment"] = ""

notes = {"CHECK # 192":"Half cow 2020",
         "CHECK # 195":"Check to MR S for dirtbike",
         "CHECK # 196":"Check to MR S for dirtbike",
         "CHECK # 132":"Arizona Department of Revenue",
         "CHECK # 131":"United States Treasury",
         "CHECK # 129":"Master Mechanical",
         "CHECK # 130":"Hunting rifle",
         "CHECK # 126":"Evelyn enrollment",
         "CASHED CHECK # 127":"Dan shoffice reno",
         "DEPOSITED OR CASHED CHECK # 128":"Shoffice electrical",
         "FID BKG SVC LLC MONEYLINE 210402 X49800767 O0TLA JOHN P CLARK":"Transfer from fidelity for tithe",
         "FRANCHISE TAX BD CASTTAXRFD 040521 XXXXX8570 CLARK III, JOHN P & HA":"Franchise tax board CASTAX",
         "CHASE CREDIT CRD EPAY 210223 5147608953 JOHN CLARK III":"2365.00 to ACSTO",
         "CHASE CREDIT CRD EPAY 210824 5467211881 JOHN CLARK III":"600 FCS, 500.38 watermans CA, 653.99 apple, 2619.25 laptop",
         "CHASE CREDIT CRD EPAY 211022 5572610997 JOHN CLARK III":"4515.75 FCS",
         "CHASE CREDIT CRD EPAY 211226 5690175783 JOHN CLARK III":"869.16 supressor, 2117.98 chair, 2435 ATC",
         "CHECK # 134":"Ranch 39 cow",
         "CHECK # 136":"Church of the resurrection",
         "CHECK # 198":"Hope Cottage",
         "PURCHASE BANK CHECK OR DRAFT":"Suppressor tax stamp check",
         "CHECK # 137":"Check Mr S for rounds",
         "CHASE CREDIT CRD EPAY 220222 5796810515 JOHN CLARK III":"Alaska airline tikets, CA memorial flight and car, Turo Alaska car"}
for idx, row in accounts_joined.iterrows():
    if row["Description"] in notes:
        note = notes[row["Description"]]
        accounts_joined.loc[idx,"comment"] = note



In [7]:
import numpy as np
import datetime as dt

columns = ["Amount","income","filtered_amount","taxes","filtered_income", "apple", "spending","full_spending","base_spending","chase","tithe","taxes_spend","shoffice"]
sum_table = accounts_joined.groupby(pd.Grouper(key='Date', freq='1M')).sum()[columns]
sum_table.loc["Average"] = sum_table[columns].iloc[:-1,:].mean()
sum_table.loc["Totals"] = sum_table[columns].iloc[:-1,:].sum()

def red(s, props=''):
    return np.where(s < 0.0, props, '')
slice_1 = ["full_spending","base_spending"]
slice_2 = ["chase"]
slice_3 = ["tithe","taxes_spend","shoffice"]
df = sum_table.style.apply(red, props='color:red;', axis=0).set_properties(**{'background-color': '#ffffb3'}, subset=slice_1).set_properties(**{'background-color': '#e8e8e8'}, subset=slice_2).set_properties(**{'background-color': '#52ffff'}, subset=slice_3).format(precision=2)
#df = df.style.apply(red, props='color:red;', axis=0, subset=slice_2).set_properties(**{'background-color': '#3E85BD'}, subset=slice_2)
display(df)
#display(sum_table)





,Amount,income,filtered_amount,taxes,filtered_income,apple,spending,full_spending,base_spending,chase,tithe,taxes_spend,shoffice
Date,,,,,,,,,,,,,
2021-02-28 00:00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3716.60,0.00,0.00,0.00,0.00
2021-03-31 00:00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,880.00,0.00,0.00,0.00,0.00
2021-04-30 00:00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2021-05-31 00:00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2021-06-30 00:00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1753.59,0.00,0.00,0.00,0.00
2021-07-31 00:00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6877.31,0.00,0.00,0.00,0.00
2021-08-31 00:00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3143.82,0.00,0.00,0.00,0.00
2021-09-30 00:00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1916.00,0.00,0.00,0.00,0.00
2021-10-31 00:00:00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,687.54,0.00,0.00,0.00,0.00


## Show month

In [3]:
month = 6; year = 2022

start_date = '{}-{}-01'.format(year,month)
if month < 12:
    end_date = '{}-{}-01'.format(year,month+1)
else:
    end_date = '{}-01-01'.format(year+1)
view_table = accounts_joined[(accounts_joined["Date"] >= start_date) & (accounts_joined["Date"] < end_date)].sort_values("Date")

view_table



,Date,Amount,Unk,CheckNum,Description,account_name,spending,income,chase,tithe,...,taxes_spend,shoffice,dirtbikes,apple,base_spend_adjustment,filtered_amount,filtered_income,full_spending,base_spending,comment
672,2022-06-01,-400.00,*,0.0,RECURRING TRANSFER TO CLARK J REF #OP0FH5Z4QV PRIME CHECKING TO COVER GAS ELECTRIC WATER IN RENTAL,savings,-400.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.00,
635,2022-06-01,400.00,*,0.0,RECURRING TRANSFER FROM CLARK J REF #OP0FH5Z4QV SAVINGS TO COVER GAS ELECTRIC WATER IN RENTAL,umbrella,0.00,400.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,400.00,400.00,0.00,0.00,
634,2022-06-01,-267.16,*,0.0,TRANSFER TO CLARK ASTRID ON 06/01 REF #PP0QGYCTBL MAY JUNE PHONE BILL,umbrella,-267.16,0.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,-267.16,0.00,-267.16,-267.16,
633,2022-06-01,-2050.88,*,0.0,DOVENMUEHLE MTG MORTG PYMT 060122 1465323044 John Clark,umbrella,-2050.88,0.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,-2050.88,0.00,-2050.88,-2050.88,
607,2022-06-03,2.00,*,0.0,ZELLE FROM TYLER REECE ON 06/03 REF # JPM999CKA4TV,utility,0.00,2.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,2.00,2.00,0.00,0.00,
700,2022-06-06,-1000.00,*,0.0,BILL PAY Rural India Ministries RECURRING No Account Number ON 06-06,primary,-1000.00,0.00,0.0,-1000.0,...,0.0,0.0,0.0,0.00,0.0,-1000.00,0.00,-1000.00,0.00,
699,2022-06-08,-127.48,*,0.0,ONLINE TRANSFER REF #IB0FK2RCSZ TO PLATINUM CARD XXXXXXXXXXXX0729 ON 06/08/22,primary,-127.48,0.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,-127.48,0.00,-127.48,-127.48,
698,2022-06-10,4636.46,*,0.0,APPLE INC. PAYROLL 489652 John P Clark III,primary,0.00,4636.46,0.0,0.0,...,0.0,0.0,0.0,4636.46,0.0,4636.46,4636.46,0.00,0.00,
697,2022-06-13,1.37,*,0.0,MOBILE DEPOSIT : REF NUMBER :506120563781,primary,0.00,1.37,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,1.37,1.37,0.00,0.00,
606,2022-06-13,-28.00,*,0.0,PURCHASE AUTHORIZED ON 06/11 VENMO* Visa Direct NY S382162814007354 CARD 4775,utility,-28.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.00,0.0,-28.00,0.00,-28.00,-28.00,


## Show Tithe

In [ ]:
monthly_tithe = 1714.79
april_bonus = 16000
oct_monthly_tithe = 1714.79
oct_bonus = 11000
tithe_rows = accounts_joined[accounts_joined["Description"].apply(func=lambda r: True if len([1 for td in tithe_desc_starts if r.startswith(td)]) > 0 else False)]
current_month = tithe_rows["Date"].max().month
current_total_tithe = current_month * monthly_tithe
if current_month >= 4:
    current_total_tithe += round(april_bonus * 0.1, 2)
if current_month >= 10:
    current_total_tithe += round(oct_bonus * 0.1, 2)
print("Current tithe total: {} Current total paid: {} Difference: {}".format(current_total_tithe, tithe_rows["Amount"].sum(), round(current_total_tithe + tithe_rows["Amount"].sum(),2)))

tithe_rows



In [ ]:
eoy_primary_spend = end_of_year[end_of_year["Payment Method"].str.startswith("Primary ck acct")]
primary_spend = primary_check[primary_check["Amount"] < 0]

display(eoy_primary_spend.groupby(pd.Grouper(key='Date', freq='1M')).sum("Amount_value"))
display(primary_spend.groupby(pd.Grouper(key='Date', freq='1M')).sum("Amount"))



In [ ]:
cashwise_card

In [ ]:
platinum_card

## Chase transactions

##### recreate chase transactions table --> looses include information

In [38]:
import pandas as pd
from os import listdir
from os.path import join

fin_dir = "/Users/johnclark/Documents/Finances"

chase_files = []
for fin_file in listdir(fin_dir):
    if fin_file.startswith("Chase1293"):
        chase_files.append(fin_file)
chase_files = sorted(chase_files)

columns=["Transaction Date", "Post Date", "Description", "Category", "Type", "Amount", "Memo"]
chase_joined = pd.DataFrame([], columns=columns)

for chase_file in chase_files:
    chase_rows = pd.read_csv(join(fin_dir, chase_file), header=0, parse_dates=[0,1])
    file_columns = set(chase_rows.columns)
    assert len(set(columns) & set(file_columns)) == len(set(columns) | set(file_columns))
    chase_joined = chase_joined.append(chase_rows)
chase_joined = chase_joined.sort_values("Post Date")
chase_joined["include"] = True
chase_joined.reset_index(drop=True, inplace=True)
chase_joined = chase_joined[~chase_joined["Description"].str.contains("Payment Thank You")].copy()
chase_joined = chase_joined[~chase_joined["Description"].str.contains("AUTOMATIC PAYMENT")].copy()




##### Load saved chase transactions

In [272]:
import pandas as pd
from os import listdir
from os.path import join

fin_dir = "/Users/johnclark/Documents/Finances"
chase_joined = pd.read_csv(join(fin_dir, "saved_modified_chase_transactions.csv"), header=0, parse_dates=[0,1])


In [274]:
chase_joined[chase_joined["include"]].sort_values("Amount", ascending=True)





,Unnamed: 0,Transaction Date,Post Date,Description,Category,Type,Amount,Memo,include
1233,1247,2021-10-15,2021-10-17,IN *FLAGSTAFF COMMUNITY C,Gifts & Donations,Sale,-4515.75,NaN,True
1345,1361,2021-11-30,2021-12-02,ARIZONA TUITITION CONNECT,Gifts & Donations,Sale,-2435.00,NaN,True
532,538,2021-02-09,2021-02-10,ACSTO,Gifts & Donations,Sale,-2365.00,NaN,True
1250,1265,2021-10-22,2021-10-24,FLAGSTAFF MOTOR COMPANY,Automotive,Sale,-909.02,NaN,True
716,724,2021-04-13,2021-04-14,SQ *WATSON CPA,Professional Services,Sale,-600.00,NaN,True
1719,1741,2022-04-04,2022-04-05,SQ *WATSON CPA,Professional Services,Sale,-600.00,NaN,True
1077,1089,2021-08-18,2021-08-19,IN *FLAGSTAFF COMMUNITY C,Gifts & Donations,Sale,-600.00,NaN,True
1276,1291,2021-11-04,2021-11-05,FLAGSTAFF MOTOR COMPANY,Automotive,Sale,-583.93,NaN,True
501,507,2021-02-01,2021-02-01,ANNUAL MEMBERSHIP FEE,Fees & Adjustments,Fee,-550.00,NaN,True
1533,1551,2022-02-01,2022-02-01,ANNUAL MEMBERSHIP FEE,Fees & Adjustments,Fee,-550.00,NaN,True


##### Save modified chase transactions

In [269]:

chase_joined.to_csv(join(fin_dir, "saved_modified_chase_transactions.csv"))


## Net Worth

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import json

with open("/Users/johnclark/Documents/Finances/networth.json") as f:
    rows = json.load(f)

net_worth = pd.DataFrame(rows)
net_worth["date"] = pd.to_datetime(net_worth["date"], format="%m/%d/%Y")
net_worth.set_index("date", drop=True, inplace=True)

nw_house_columns = ["Wells cash", "Wells credit", "chase credit", "wells brokerage", "fidelity", "etrade", "etrade stock","empower hannah", "empower john", "House value", "mortgage", "tithe"]
net_worth["total with house"] = net_worth[nw_house_columns].sum(axis=1)
nw_columns = ["Wells cash", "Wells credit", "chase credit", "wells brokerage", "fidelity", "etrade", "etrade stock","empower hannah", "empower john", "tithe"]
net_worth["total"] = net_worth[nw_columns].sum(axis=1)
net_worth["difference"] = 0.0
for row_idx in range(1,len(net_worth)):
    row_name = net_worth.iloc[row_idx].name
    net_worth.loc[row_name,"difference"] = net_worth.iloc[row_idx]["total"] - net_worth.iloc[row_idx-1]["total"]

display(net_worth)
fig = plt.figure()
ax1 = fig.add_subplot(311)
ax2 = fig.add_subplot(312)
ax3 = fig.add_subplot(313)
_ = net_worth[nw_columns].plot(legend=True, figsize=(30,30), ax=ax1)
_ = net_worth[["total with house","total"]].plot(legend=True, figsize=(30,30), ax=ax2)
_ = net_worth["difference"].plot.bar(legend=True, figsize=(30,30), ax=ax3)
ax1.grid(color='gray', linewidth=1, axis='both', alpha=0.5)
ax2.grid(color='gray', linewidth=1, axis='both', alpha=0.5)
ax2.axvline(x=dt.date(2022,2,7), color='red', linestyle='-', label='biked 75K')
ax2.axvline(x=dt.date(2022,4,6), color='green', linestyle='-', label='taxes 58K')
ax2.axvline(x=dt.date(2022,2,23), color='purple', linestyle='-', label='alaska 13K')
ax2.axvline(x=dt.date(2022,6,29), color='blue', linestyle='-', label='alaska 12K')
ax2.legend()
ax3.grid(color='gray', linewidth=1, axis='both', alpha=0.5)



In [ ]:
75+58+13